# Vehicle Routing Problem using Google OR-Tools

vehicle_capacities: 2 大 1 小, 容量不確定

In [78]:
# Import modules
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# record time/space complexity
import time 
import tracemalloc

In [75]:
# Model
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["time_matrix"] = [
      [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
      [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
      [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
      [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
      [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
      [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
      [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
      [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
      [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
      [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
      [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
      [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
      [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
      [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
      [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
      [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
      [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data["vehicle_capacities"] = [15, 15, 15, 15]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data

def create_result_model():
  result = {}
  result["routes"] = [],
  result["total_time"] = [],
  result["cumul_dist"] = [],
  result["algorithm"] = [],
  result["running_time"] = [],
  result["memory_usage"] = []
  return result

In [66]:
# View
def print_distance_solution(data, manager, routing, solution):
  """Prints solution on console."""
  print(f"Objective: {solution.ObjectiveValue()}")
  total_dists = []
  total_distance = 0
  total_load = 0
  for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    plan_output = f"Route for vehicle {vehicle_id}:\n"
    route_distance = 0
    route_load = 0
    while not routing.IsEnd(index):
      node_index = manager.IndexToNode(index)
      route_load += data["demands"][node_index]
      plan_output += f" {node_index} Load({route_load}) -> "
      previous_index = index
      index = solution.Value(routing.NextVar(index))
      route_distance += routing.GetArcCostForVehicle(
          previous_index, index, vehicle_id
      )
    plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
    plan_output += f"Distance of the route: {route_distance}m\n"
    plan_output += f"Load of the route: {route_load}\n"
    print(plan_output)
    total_distance += route_distance
    total_load += route_load
    total_dists.append(total_distance)
  print(f"Total distance of all routes: {total_distance}m")
  print(f"Total load of all routes: {total_load}")
  return total_dists

def print_time_solution(routes, cumul_data):
  """Print the solution."""
  total_time = 0
  route_str = ''
  for i, route in enumerate(routes):
    route_str += 'Route ' + str(i) + ':\n'
    start_time = cumul_data[i][0][0]
    end_time = cumul_data[i][0][1]
    route_str += '  ' + str(route[0]) + \
                 ' Time(' + str(start_time) + ', ' + str(end_time) + ')'
    for j in range(1, len(route)):
      start_time = cumul_data[i][j][0]
      end_time = cumul_data[i][j][1]
      route_str += ' -> ' + str(route[j]) + \
                   ' Time(' + str(start_time) + ', ' + str(end_time) + ')'
    route_str += f'\n  Route time: {start_time}min\n\n'
    total_time += cumul_data[i][len(route) - 1][0]
  route_str += f'Total time: {total_time}min'
  print(route_str)
  return total_time

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.
  cumul_data = []
  for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      dim_var = dimension.CumulVar(index)
      route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
  return cumul_data

In [95]:
# Controller
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        0,  # allow waiting time
        60,  # maximum time per vehicle
        True,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    algo = "greedy"
    # 變更演算法: GREEDY_DESCENT, SIMULATED_ANNEALING, TABU_SEARCH
    match algo:
        case "greedy":
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT
            )
        case "simulated_anneling":
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING
            )
        case "tabu_search":
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH
            )
        case _:
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
            )
    # search_parameters.time_limit.seconds = 5
    search_parameters.solution_limit = 200
    search_parameters.log_search = False

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        result["algorithm"] = algo
        result["routes"] = get_routes(solution, routing, manager)
        result["cumul_data"] = get_cumul_data(solution, routing, time_dimension)
        result["cumul_dist"] = print_distance_solution(data, manager, routing, solution)
        result["total_time"] = print_time_solution(result["routes"], result["cumul_data"])
        return result

if __name__ == "__main__":
    start_time = time.time()
    tracemalloc.start()
    result = create_result_model()
    main()
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    result["running_time"] = round(end_time - start_time, 4)
    result["memory_usage"] = round(current / 1024, 4)

Objective: 72
Route for vehicle 0:
 0 Load(0) ->  1 Load(1) ->  4 Load(5) ->  3 Load(7) ->  15 Load(15) ->  0 Load(15)
Distance of the route: 24m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  16 Load(12) ->  10 Load(14) ->  2 Load(15) ->  0 Load(15)
Distance of the route: 24m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  7 Load(8) ->  13 Load(12) ->  11 Load(13) ->  12 Load(15) ->  0 Load(15)
Distance of the route: 13m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  8 Load(8) ->  6 Load(12) ->  5 Load(14) ->  9 Load(15) ->  0 Load(15)
Distance of the route: 11m
Load of the route: 15

Total distance of all routes: 72m
Total load of all routes: 60
Route 0:
  0 Time(0, 0) -> 1 Time(6, 6) -> 4 Time(8, 8) -> 3 Time(9, 9) -> 15 Time(15, 15) -> 0 Time(24, 24)
  Route time: 24min

Route 1:
  0 Time(0, 0) -> 14 Time(5, 5) -> 16 Time(7, 7) -> 10 Time(11, 11) -> 2 Time(15, 15) -> 0 Time(24, 24)
  Route time: 24min

Route 2:
  0 Time(0, 0) -> 7 T

In [98]:
result

{'routes': [[0, 1, 4, 3, 15, 0],
  [0, 14, 16, 10, 2, 0],
  [0, 7, 13, 11, 12, 0],
  [0, 8, 6, 5, 9, 0]],
 'total_time': 72,
 'cumul_dist': [24, 48, 61, 72],
 'algorithm': 'greedy',
 'running_time': 0.072,
 'memory_usage': 7.9727,
 'cumul_data': [[[0, 0], [6, 6], [8, 8], [9, 9], [15, 15], [24, 24]],
  [[0, 0], [5, 5], [7, 7], [11, 11], [15, 15], [24, 24]],
  [[0, 0], [2, 2], [5, 5], [8, 8], [9, 9], [13, 13]],
  [[0, 0], [3, 3], [5, 5], [7, 7], [9, 9], [11, 11]]]}